# Measurement Error Mitigation

In [1]:
#import of the functions I defined in my_rem
from my_rem import * 

In [3]:
# Set up the readout error mitigation by constructing 
# the mitigator based on the number of qubits, the number of shots, and the readout noise model

n_qubits = 2
shots = 100000
error_probability = 0.1
readout_noise = get_readout_noise(error_probability)

mitigator = generate_mitigator(n_qubits, shots, readout_noise)

States:
['00', '01', '10', '11']
00 becomes {'11': 986, '01': 9143, '10': 8877, '00': 80994}
01 becomes {'10': 1049, '00': 8953, '11': 8963, '01': 81035}
10 becomes {'01': 1023, '11': 8990, '00': 8927, '10': 81060}
11 becomes {'00': 1078, '10': 9054, '01': 8933, '11': 80935}

Confusion Matrix:
[[0.809940 0.091430 0.088770 0.009860]
 [0.089530 0.810350 0.010490 0.089630]
 [0.089270 0.010230 0.810600 0.089900]
 [0.010780 0.089330 0.090540 0.809350]] 

Mitigator:
[[1.265497 -0.142774 -0.138501 0.015778]
 [-0.139576 1.265033 0.014553 -0.140010]
 [-0.139171 0.015222 1.264384 -0.140434]
 [0.014119 -0.139426 -0.141205 1.266512]]


In [7]:
# Now we define an arbitrary circuit and apply the mitigation 

shots = 100000

qc = QuantumCircuit(n_qubits,n_qubits)
qc.h(0)
qc.cx(0,1) 
qc.measure(qc.qregs[0],qc.cregs[0])
#print(qc)
simulator = AerSimulator(noise_model = readout_noise) # Assuming that there is no error in the circuit, just in the measurement
counts = simulator.run(qc,shots=shots).result().get_counts(qc)
# Normalize the counts to get probabilities
probabilities = {k: v / shots for k, v in counts.items()}

# Sort the probabilities dictionary by its keys (binary strings converted to integers)
print("Empirical probabilities:", (probability_vector:=np.array([probability for state, probability in sorted(probabilities.items(), key=lambda x: int(x[0], 2))])))

# Apply the mitigation to the probabilities
print("Mitigated quasi-probabilities:", (mitigated_probabilities := np.dot(mitigator, probability_vector)))

# Closest positive quasi-probabilities via mitiq function
print("Closest positive probabilities:", (closest_probabilities := closest_positive_distribution(mitigated_probabilities)))

Empirical probabilities: [0.409600 0.090010 0.089860 0.410530]
Mitigated quasi-probabilities: [0.499528 0.000525 0.000331 0.500486]
Closest positive probabilities: [0.499335 0.000271 0.000162 0.500232]


In [46]:
# If we are interested in generating some samples from the mitigated distribution...
samples = 10
bitstrings = sample_probability_vector(closest_probabilities, samples)
print(bitstrings)

[[1, 1], [1, 1], [0, 0], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [0, 0], [1, 1]]
